# Eat Safe, Love

## Notebook Set Up

In [2]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our database
collections = db.list_collection_names()
for coll_name in collections:
    print(coll_name)

establishments


In [6]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [33]:
# Find the establishments with a hygiene score of 20
hygiene_20_results = list(establishments.find({"scores.Hygiene": 20}))

# Use count_documents to display the number of documents in the result
count_hygiene_20 = establishments.count_documents({"scores.Hygiene": 20})

# Display the first document in the results using pprint
first_hygiene_20 = establishments.find_one({"scores.Hygiene": 20})
pprint(first_hygiene_20)

{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': '0',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64cf17a2705a12a8b30ada05'),
 'geocode': {'latitude': Decimal128('50.769705'),
             'longitude': Decimal128('0.27694')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'}],
 'meta': 

In [42]:
# Convert the result to a Pandas DataFrame
import pandas as pd
hygiene_20_df = pd.DataFrame(hygiene_20_results)

# Display the number of rows in the DataFrame
print(len(hygiene_20_df))

# Display the first 10 rows of the DataFrame
print(hygiene_20_df.head(10))

41
                        _id   FHRSID  ChangesByServerID   
0  64cf17a2705a12a8b30ada05   110681                  0  \
1  64cf17a2705a12a8b30add88   612039                  0   
2  64cf17a2705a12a8b30ae090   730933                  0   
3  64cf17a2705a12a8b30ae282   172735                  0   
4  64cf17a2705a12a8b30ae28b   172953                  0   
5  64cf17a2705a12a8b30aec2c   512854                  0   
6  64cf17a2705a12a8b30aee59  1537089                  0   
7  64cf17a3705a12a8b30b0375   155648                  0   
8  64cf17a3705a12a8b30b07bf  1012883                  0   
9  64cf17a3705a12a8b30b0fce   644109                  0   

  LocalAuthorityBusinessID               BusinessName   
0                     4029        The Chase Rest Home  \
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza   
4             PI/000024532              Golden Palace   
5    

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [109]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {
    "$or": [
        {"AddressLine2": "London"},
        {"AddressLine3": "London"}
    ],
    "RatingValue": {"$in": ["4", "5"]}
}

# Use count_documents to display the number of documents in the result
# count_london_high_rating = establishments.count_documents({"RatingValue": "4"})
# count_london_high_rating = establishments.count_documents({"RatingValue": {"$in": ["4", "5"]}})
count_london_high_rating = establishments.count_documents(query)

# Display the first document in the results using pprint
# first_london_high_rating = establishments.find_one(query)
pprint(count_london_high_rating)

1103


In [114]:
# Convert the result to a Pandas DataFrame
london_address_rating_results = list(establishments.find(query))
london_address_rating_df = pd.DataFrame(london_address_rating_results)

# Display the number of rows in the DataFrame
print(len(london_address_rating_df))

# Display the first 10 rows of the DataFrame
print(london_address_rating_df.head(10))

1103
                        _id   FHRSID  ChangesByServerID   
0  64cf17a3705a12a8b30b17a1   294900                  0  \
1  64cf17a3705a12a8b30b29dd   329113                  0   
2  64cf17a3705a12a8b30b29de   329114                  0   
3  64cf17a3705a12a8b30b2b41  1402880                  0   
4  64cf17a3705a12a8b30b2be4   975472                  0   
5  64cf17a3705a12a8b30b2be6  1262185                  0   
6  64cf17a3705a12a8b30b2d2e  1408389                  0   
7  64cf17a3705a12a8b30b2d3f  1438466                  0   
8  64cf17a3705a12a8b30b2d49  1156030                  0   
9  64cf17a3705a12a8b30b2d54   687040                  0   

  LocalAuthorityBusinessID                                       BusinessName   
0             PI/000019066                                         Mv Valulla  \
1         03260/0075/0/000                              The Croft Day Nursery   
2         03260/2001/2/000                     Belvedere Sports & Social Club   
3              21/002

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [130]:
# # Search within 0.01 degree on either side of the latitude and longitude.
# import math

# # Fetch the geocoordinates of "Penang Flavours" from the list
# penang_flavours = next((e for e in establishments if e["BusinessName"] == "Penang Flavours"), None)
# if not penang_flavours:
#     raise ValueError("Penang Flavours not found in the list")

# latitude_pf = float(str(penang_flavours["geocode"]["latitude"]))
# longitude_pf = float(str(penang_flavours["geocode"]["longitude"]))

# # Filter establishments with a RatingValue of "5"
# all_establishments = [e for e in establishments if e["RatingValue"] == "5"]

# # Calculate the Euclidean distance for each establishment
# for est in all_establishments:
#     latitude_est = float(str(est["geocode"]["latitude"]))
#     longitude_est = float(str(est["geocode"]["longitude"]))
    
#     distance = math.sqrt((latitude_pf - latitude_est)**2 + (longitude_pf - longitude_est)**2)
#     est['distance'] = distance

# # Sort by distance and then by hygiene score
# sorted_establishments = sorted(
#     all_establishments, 
#     key=lambda x: (
#         x.get('distance') if x.get('distance') is not None else float('inf'), 
#         x.get('scores', {}).get('Hygiene') if x.get('scores', {}).get('Hygiene') is not None else float('inf')
#     )
# )

# # Limit to top 5
# top_5 = sorted_establishments[:5]

# # Print the results
# for r in top_5:
#     print(r)

import math

# Fetch the geocoordinates of "Penang Flavours" from the list
penang_flavours = next((e for e in establishments if e["BusinessName"] == "Penang Flavours"), None)
if not penang_flavours:
    raise ValueError("Penang Flavours not found in the list")

latitude_pf = float(str(penang_flavours["geocode"]["latitude"]))
longitude_pf = float(str(penang_flavours["geocode"]["longitude"]))

degree_search = 0.01

# Filter establishments with a RatingValue of "5" and within the range of coordinates
all_establishments = [
    e for e in establishments 
    if e["RatingValue"] == "5"
    and float(str(e["geocode"]["latitude"])) >= latitude_pf - degree_search
    and float(str(e["geocode"]["latitude"])) <= latitude_pf + degree_search
    and float(str(e["geocode"]["longitude"])) >= longitude_pf - degree_search
    and float(str(e["geocode"]["longitude"])) <= longitude_pf + degree_search
]


# Calculate the Euclidean distance for each establishment
for est in all_establishments:
    latitude_est = float(str(est["geocode"]["latitude"]))
    longitude_est = float(str(est["geocode"]["longitude"]))
    
    distance = math.sqrt((latitude_pf - latitude_est)**2 + (longitude_pf - longitude_est)**2)
    est['distance'] = distance

# Sort by distance and then by hygiene score
sorted_establishments = sorted(
    all_establishments, 
    key=lambda x: (
        x.get('distance') if x.get('distance') is not None else float('inf'), 
        x.get('scores', {}).get('Hygiene') if x.get('scores', {}).get('Hygiene') is not None else float('inf')
    )
)

# Limit to top 5
top_5 = sorted_establishments[:5]

# Print the results
for r in top_5:
    print(r)



{'_id': ObjectId('64cf1f54364f316ceeadbc87'), 'FHRSID': 123456, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/0000XXXXX', 'BusinessName': 'Penang Flavours', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': '123 Greenwich Street', 'AddressLine2': 'City Centre', 'AddressLine3': 'Greenwich', 'AddressLine4': '', 'PostCode': 'CT1 2YZ', 'Phone': '01234 567890', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2023-08-05T00:00:00', 'LocalAuthorityCode': '182', 'LocalAuthorityName': 'Kent', 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk', 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': Decimal128('1.076614'), 'latitude': Decimal128('51.278708')}, 'RightToReply': '', 'Distance': 0, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'return

In [134]:
# Convert result to Pandas DataFrame
df_establishments = pd.DataFrame(sorted_establishments)
# df_establishments.head(10)

,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,LocalAuthorityEmailAddress,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending,meta,links,distance
0,64cf1f54364f316ceeadbc87,123456,0,PI/0000XXXXX,Penang Flavours,Restaurant/Cafe/Canteen,1,123 Greenwich Street,City Centre,Greenwich,...,health@royalgreenwich.gov.uk,"{'Hygiene': 5, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 1.076614, 'latitude': 51.278708}",,0.000000,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings....",0.000000
1,64d1af1f364f316ceeadbc88,123456,0,PI/0000XXXXX,Penang Flavours,Restaurant/Cafe/Canteen,1,123 Greenwich Street,City Centre,Greenwich,...,health@royalgreenwich.gov.uk,"{'Hygiene': 5, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': '1.076614', 'latitude': '51.2787...",,0.000000,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings....",0.000000
2,64d1c7fb364f316ceeadbc8a,123456,0,PI/0000XXXXX,Penang Flavours,Restaurant/Cafe/Canteen,1,123 Greenwich Street,City Centre,Greenwich,...,health@royalgreenwich.gov.uk,"{'Hygiene': 5, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': '1.076614', 'latitude': '51.2787...",,0.000000,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings....",0.000000
3,64d1c9a8364f316ceeadbc8c,123456,0,PI/0000XXXXX,Penang Flavours,Restaurant/Cafe/Canteen,1,123 Greenwich Street,City Centre,Greenwich,...,health@royalgreenwich.gov.uk,"{'Hygiene': 5, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': '1.076614', 'latitude': '51.2787...",,0.000000,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings....",0.000000
4,64cf17a2705a12a8b30aca9f,1103638,0,CP/18/00181,The Mooring,Restaurant/Cafe/Canteen,1,Water Lane Cafe,Water Lane,Canterbury,...,,"{'Hygiene': 5, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 1.07705497741699, 'latitude': 51...",,4603.383987,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings....",0.000494
5,64cf17a2705a12a8b30acad4,1103379,0,81749,Caterlink - St Peters Methodist Primary School,School/college/university,7845,St Peter's Grove,Canterbury,,...,,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 1.07625102996826, 'latitude': 51...",,4603.468977,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings....",0.001315
6,64cf17a2705a12a8b30acad3,1102842,0,73772,St Peters Breakfast And After School Club,School/college/university,7845,St Peters Methodist Primary School,St Peter's Grove,Canterbury,...,,"{'Hygiene': 5, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 1.07625102996826, 'latitude': 51...",,4603.468977,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings....",0.001315
7,64cf17a2705a12a8b30aca9c,1103038,0,77542,Greyfriars Lodge,Hotel/bed & breakfast/guest house,7842,6 Stour Street,Canterbury,,...,,"{'Hygiene': 5, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': 1.07794201374054, 'latitude': 51...",,4603.382275,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings....",0.001453
8,64cf17a2705a12a8b30aca9b,1340744,0,CP/21/00029,Fond Coffee,Restaurant/Cafe/Canteen,1,1-2 Jewry Lane,Canterbury,Kent,...,,"{'Hygiene': 5, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 1.07795298099518, 'latitude': 51...",,4603.381447,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings....",0.001458
9,64cf17a2705a12a8b30aca8a,1128225,0,CP/19/00070,The Foundry Brew Pub,Pub/bar/nightclub,7843,77 Stour Street,Canterbury,Kent,...,,"{'Hygiene': 5, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': 1.

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [145]:
# Define the MongoDB collection again for some reason!
establishments_mongo = db['establishments']

# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {
        "_id": "$LocalAuthorityName", 
        "count": {"$sum": 1}
    }},
    {"$sort": {"count": -1}}
]

# Execute the aggregation pipeline on the uniquely named MongoDB collection
results = establishments_mongo.aggregate(pipeline)

# Convert the results to a list
results_list = list(results)

# Print the number of documents in the result
print(len(results_list))

# Print the first 10 results
for r in results_list[:10]:
    print(r)


55
{'_id': 'Thanet', 'count': 1130}
{'_id': 'Greenwich', 'count': 882}
{'_id': 'Maidstone', 'count': 713}
{'_id': 'Newham', 'count': 711}
{'_id': 'Swale', 'count': 686}
{'_id': 'Chelmsford', 'count': 680}
{'_id': 'Medway', 'count': 672}
{'_id': 'Bexley', 'count': 607}
{'_id': 'Southend-On-Sea', 'count': 586}
{'_id': 'Tendring', 'count': 542}


In [147]:
# Convert the result to a Pandas DataFrame
df_results = pd.DataFrame(results_list)

# Display the number of rows in the DataFrame
num_rows = df_results.shape[0]
print(f"Number of rows in the DataFrame: {num_rows}")

# Display the first 10 rows of the DataFrame
df_results.head(10)

Number of rows in the DataFrame: 55


,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
